**Problem Statement:**

Design a car’s double-wishbone suspension system by selecting key mechanical parameters to balance ride comfort, handling, durability, and manufacturing cost.

Input Variables:

| Variable              | Description                                  | Units       |
| --------------------- | -------------------------------------------- | ----------- |
| `spring_stiffness`    | Spring stiffness                             | N/m         |
| `damping_coefficient` | Damping coefficient                          | Ns/m        |
| `unsprung_mass`       | Mass not supported by suspension             | kg          |
| `lower_arm_length`    | Lower wishbone arm length                    | m           |
| `upper_arm_angle_deg` | Upper wishbone angle                         | degrees     |
| `tire_stiffness`      | Stiffness of the tire                        | N/m         |

Objectives:

| Objective            | Description                                |
| -------------------- | ------------------------------------------ |
| `ride_discomfort`    | Vertical acceleration-based discomfort     |
| `handling_quality`   | Tire-road contact effectiveness (maximize) |
| `fatigue_index`      | Durability index (lower = better)          |
| `manufacturing_cost` | Estimated total cost of system             |




In [11]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from pymoo.core.mixed import  MixedVariableSampling, MixedVariableMating,MixedVariableDuplicateElimination
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.variable import Real, Integer, Choice
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor
from pymoo.operators.survival.rank_and_crowding.metrics import calc_crowding_distance
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.algorithms.moo.nsga3 import NSGA3

**Reading the Synthetic generated data**

In [3]:
df_suspension = pd.read_csv('df_suspension.csv')

**Buliding ML models between inputs and the objectives**

In [4]:
# Features and targets
X = df_suspension[[
    "spring_stiffness",
    "damping_coefficient",
    "unsprung_mass",
    "lower_arm_length",
    "upper_arm_angle_deg",
    "tire_stiffness"
]]

y_targets = {
    "ride_discomfort": df_suspension["ride_discomfort"],
    "handling_quality": df_suspension["handling_quality"],
    "fatigue_index": df_suspension["fatigue_index"],
    "manufacturing_cost": df_suspension["manufacturing_cost"]
}

# Split data
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

models = {}
metrics = {}

for target_name, y in y_targets.items():
    y_train, y_test = train_test_split(y, test_size=0.2, random_state=42)
    
    model = XGBRegressor(
        n_estimators=100,
        max_depth=4,
        learning_rate=0.1,
        objective='reg:squarederror',
        verbosity=0
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    models[target_name] = model
    metrics[target_name] = {
        "R2": r2_score(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred))
    }

metrics


{'ride_discomfort': {'R2': 0.9958035869943223, 'RMSE': 0.03793099437663878},
 'handling_quality': {'R2': 0.9964683134670561, 'RMSE': 7.967660799863099},
 'fatigue_index': {'R2': 0.9225197984448338, 'RMSE': 1.0888512322917472},
 'manufacturing_cost': {'R2': 0.9947853624863139, 'RMSE': 224.65174661135265}}

**Search Space with bounds & Objective:**

In [5]:
continuous = {
    'spring_stiffness': (20000, 100000),       
    'damping_coefficient': (500, 5000),       
    'lower_arm_length': (0.2, 0.5),            
    'tire_stiffness': (100000, 300000)         
}

integer = {
    'unsprung_mass': [30, 40, 50, 60, 70, 80],                      
    'upper_arm_angle_deg': [5, 10, 15, 20, 25],                    
}
objectives = {
    'ride_discomfort': 'minimize',
    'handling_quality': 'maximize',
    'fatigue_index': 'minimize',
    'manufacturing_cost': 'minimize'
} 

**Feature order - order in which variables used for training**

In [6]:
feature_order = list(X.columns)

**Defining Mixed variable Optmization Problem**

In [35]:
variables_re

{'spring_stiffness': <pymoo.core.variable.Real at 0x19b333a06a0>,
 'damping_coefficient': <pymoo.core.variable.Real at 0x19b332244f0>,
 'unsprung_mass': <pymoo.core.variable.Choice at 0x19b333a26b0>,
 'lower_arm_length': <pymoo.core.variable.Real at 0x19b33224460>,
 'upper_arm_angle_deg': <pymoo.core.variable.Choice at 0x19b33225a80>,
 'tire_stiffness': <pymoo.core.variable.Real at 0x19b33224400>}

In [8]:
real_variables = {}
choice_variables={}
for key,value in integer.items():
    choice_variables[key] = Choice(options=value)

for key,value in continuous.items():
    real_variables[key] =  Real(bounds=(value[0],value[1]))

variables = real_variables | choice_variables
variables_re =  {key: variables[key] for key in feature_order if key in variables}


class MultiObjectiveMixedVariableProblem(ElementwiseProblem):

    def __init__(self, model,variables,objective,**kwargs):
        self.model = model
        self.variables = variables
        self.objective = objective
        n_obj = len(objective)
        super().__init__(vars=self.variables, n_obj=n_obj, **kwargs)

    def _evaluate(self, X, out, *args, **kwargs):
        X_values = np.array(list(X.values())).reshape(1, -1)
        predictions = []
        for key,value in self.objective.items():
            if value == 'maximize':
                y_pred = -1*(self.model[key].predict(X_values))
                predictions.append(y_pred[0])
            else:
                y_pred = self.model[key].predict(X_values)
                predictions.append(y_pred[0])
        out["F"] = predictions
        

optimization_problem = MultiObjectiveMixedVariableProblem(
        models,
        variables_re,objectives
    )


**NSGA2 Implementation**

In [18]:
algorithm = NSGA2(
    pop_size=100,
    sampling=MixedVariableSampling(),
    mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
    eliminate_duplicates=MixedVariableDuplicateElimination(),
)
res = minimize(optimization_problem,
                  algorithm,
                  seed=42,
                  verbose=False,
                  save_history=True)

In [31]:
X_feasible = res.X  
F_feasible = res.F
nds = NonDominatedSorting()
fronts = nds.do(F_feasible)  


F_non_dominated = F_feasible[fronts[0]]
X_non_dominated = X_feasible[fronts[0]]


cd = calc_crowding_distance(F_non_dominated)


top_indices = np.argsort(-cd)[:5]  
best_X = X_non_dominated[top_indices]
best_F = F_non_dominated[top_indices]

for count,fitness in enumerate(best_F):
    for pos,val in enumerate(fitness):
        if list(objectives.values())[pos]=='maximize':
            fitness[pos] = -1*fitness[pos] 
        else:
            pass
optimal_inputs = [x for x in best_X]
optimal_objectives = [dict(zip(list(objectives.keys()), row)) for row in best_F]

**Optimal inputs**

In [33]:
optimal_inputs

[{'spring_stiffness': 22845.088300568576,
  'damping_coefficient': 1661.382110556341,
  'lower_arm_length': 0.3751871366603923,
  'tire_stiffness': 145878.94914098992,
  'unsprung_mass': 40,
  'upper_arm_angle_deg': 20},
 {'spring_stiffness': 20599.176469335,
  'damping_coefficient': 4582.945287336773,
  'lower_arm_length': 0.2965728098219272,
  'tire_stiffness': 160138.42715011095,
  'unsprung_mass': 80,
  'upper_arm_angle_deg': 15},
 {'spring_stiffness': 29763.058787582304,
  'damping_coefficient': 4831.0128272395,
  'unsprung_mass': 70,
  'lower_arm_length': 0.21232025503036275,
  'upper_arm_angle_deg': 5,
  'tire_stiffness': 239082.22462132951},
 {'spring_stiffness': 95910.84298026667,
  'damping_coefficient': 2019.2682713163258,
  'unsprung_mass': 50,
  'lower_arm_length': 0.2583140101984619,
  'upper_arm_angle_deg': 25,
  'tire_stiffness': 295580.1337508542},
 {'spring_stiffness': 20441.769369888192,
  'damping_coefficient': 2804.418762346764,
  'unsprung_mass': 80,
  'lower_arm_

**Optimal Objectives**

In [34]:
optimal_objectives 

[{'ride_discomfort': 0.6758543848991394,
  'handling_quality': 173.47769165039062,
  'fatigue_index': 7.150527000427246,
  'manufacturing_cost': 6564.13134765625},
 {'ride_discomfort': 0.6733095645904541,
  'handling_quality': 170.3580780029297,
  'fatigue_index': 2.831373691558838,
  'manufacturing_cost': 6830.466796875},
 {'ride_discomfort': 0.4409847557544708,
  'handling_quality': 259.5946044921875,
  'fatigue_index': 0.0509452261030674,
  'manufacturing_cost': 13392.076171875},
 {'ride_discomfort': 1.9293001890182495,
  'handling_quality': 665.8521728515625,
  'fatigue_index': 3.5059852600097656,
  'manufacturing_cost': 19507.625},
 {'ride_discomfort': 0.2753681540489197,
  'handling_quality': 181.94676208496094,
  'fatigue_index': 1.112094521522522,
  'manufacturing_cost': 12729.9931640625}]

**NSGA3 Implementation**

In [12]:
n_obj = len(objectives)

ref_dirs = get_reference_directions("das-dennis", n_obj, n_partitions=10)
algorithm = NSGA3(ref_dirs=ref_dirs,pop_size=len(ref_dirs),sampling=MixedVariableSampling(),
                    mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
                    eliminate_duplicates=MixedVariableDuplicateElimination())

res = minimize(optimization_problem,
                  algorithm,
                  seed=42,
                  verbose=False,
                  save_history=True)

In [14]:
X_feasible = res.X  
F_feasible = res.F
nds = NonDominatedSorting()
fronts = nds.do(F_feasible)  


F_non_dominated = F_feasible[fronts[0]]
X_non_dominated = X_feasible[fronts[0]]


cd = calc_crowding_distance(F_non_dominated)


top_indices = np.argsort(-cd)[:5]  
best_X = X_non_dominated[top_indices]
best_F = F_non_dominated[top_indices]

for count,fitness in enumerate(best_F):
    for pos,val in enumerate(fitness):
        if list(objectives.values())[pos]=='maximize':
            fitness[pos] = -1*fitness[pos] 
        else:
            pass
optimal_inputs = [x for x in best_X]
optimal_objectives = [dict(zip(list(objectives.keys()), row)) for row in best_F]

**Optimal Inputs**

In [15]:
optimal_inputs

[{'spring_stiffness': 78800.10981361875,
  'damping_coefficient': 566.9285235664543,
  'lower_arm_length': 0.32223272147580856,
  'tire_stiffness': 252811.8510971255,
  'unsprung_mass': 40,
  'upper_arm_angle_deg': 10},
 {'spring_stiffness': 97332.38552349356,
  'damping_coefficient': 2999.4426752118234,
  'unsprung_mass': 80,
  'lower_arm_length': 0.4710091437908075,
  'upper_arm_angle_deg': 5,
  'tire_stiffness': 279606.10978999734},
 {'spring_stiffness': 39011.003519391976,
  'damping_coefficient': 4157.598052658761,
  'unsprung_mass': 60,
  'lower_arm_length': 0.23427137777611906,
  'upper_arm_angle_deg': 10,
  'tire_stiffness': 227823.98756310006},
 {'spring_stiffness': 23588.352399858635,
  'damping_coefficient': 1395.402107059028,
  'lower_arm_length': 0.39234492954774225,
  'tire_stiffness': 275908.6161941937,
  'unsprung_mass': 30,
  'upper_arm_angle_deg': 25},
 {'spring_stiffness': 98950.95492804138,
  'damping_coefficient': 3390.8867519284204,
  'unsprung_mass': 30,
  'lower

**Optimal Objectives**

In [16]:
optimal_objectives

[{'ride_discomfort': 2.5618908405303955,
  'handling_quality': 426.17822265625,
  'fatigue_index': 33.1205940246582,
  'manufacturing_cost': 9412.58203125},
 {'ride_discomfort': 1.222818374633789,
  'handling_quality': 713.8765258789062,
  'fatigue_index': 3.7846755981445312,
  'manufacturing_cost': 18948.70703125},
 {'ride_discomfort': 0.681006133556366,
  'handling_quality': 330.2305908203125,
  'fatigue_index': 0.3334009647369385,
  'manufacturing_cost': 13547.740234375},
 {'ride_discomfort': 0.7179917097091675,
  'handling_quality': 184.72923278808594,
  'fatigue_index': 8.977934837341309,
  'manufacturing_cost': 6568.3896484375},
 {'ride_discomfort': 3.1853697299957275,
  'handling_quality': 516.764892578125,
  'fatigue_index': 2.684833288192749,
  'manufacturing_cost': 10641.8681640625}]